In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import mysql.connector

C:\Users\nguye\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_train = pd.read_csv('C:/Python/Marketing-Job-Data-Scraping/SQL/step-7-Industry Classification/training-data.csv')
df_train

,Type of Industry,Name
0,Advertising and Marketing,Digital Advertising
1,Advertising and Marketing,Content Marketing
2,Advertising and Marketing,Social Media Marketing
3,Advertising and Marketing,Influencer Marketing
4,Advertising and Marketing,Search Engine Optimization (SEO) Services
...,...,...
1264,Manufacturing,Telecommunication Equipment Manufacturing
1265,Manufacturing,Office Supplies and Stationery Manufacturing
1266,Manufacturing,Solar Panel and Renewable Energy Equipment Man...
1267,Manufacturing,Power Generation and Distribution Equipment Ma...


In [3]:
# Feature extraction - Convert industry name into numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(df_train['Name'])

# Target labels
y_train = df_train['Type of Industry']

# Train a Support Vector Machine (SVM) classifier
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [4]:
# Test: Categorize new industry
new = [
    'Insurance'
]
X_new = tfidf_vectorizer.transform(new)
predicted_categories = classifier.predict(X_new)

for industry, category in zip(new, predicted_categories):
    print(f"Industry: {industry} --> Group: {category}")

Industry: Insurance --> Group: Financial Services


In [6]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="jobs-scraping.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user="root",
    password="123456789",
    database="job_scraping_database"
)

cursor = mydb.cursor()

In [7]:
select_query = "Select * from industry"
cursor.execute(select_query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns = cursor.column_names)

In [8]:
df

,id,industry,industry_group,industry_count
0,1,Warehousing and Storage,None,2
1,2,Environmental Services,None,13
2,3,Events Services,None,18
3,4,Medical Equipment Manufacturing,None,9
4,5,Manufacturing,None,53
...,...,...,...,...
191,192,Computer Hardware Manufacturing,None,3
192,193,Cosmetics,None,1
193,194,Mechanical Or Industrial Engineering,None,1
194,195,Mobile Computing Software Products,None,1


In [9]:
new_industry = df['industry']

X1 = tfidf_vectorizer.transform(new_industry)

predicted_categories = classifier.predict(X1)
df['industry_group'] = predicted_categories

In [10]:
df

,id,industry,industry_group,industry_count
0,1,Warehousing and Storage,Others,2
1,2,Environmental Services,Environmental and Energy,13
2,3,Events Services,Travel and Tourism,18
3,4,Medical Equipment Manufacturing,Manufacturing,9
4,5,Manufacturing,Manufacturing,53
...,...,...,...,...
191,192,Computer Hardware Manufacturing,Manufacturing,3
192,193,Cosmetics,Travel and Tourism,1
193,194,Mechanical Or Industrial Engineering,Others,1
194,195,Mobile Computing Software Products,Business and Information Technology,1


In [12]:
for index, row in df.iterrows():
    industry = row["industry"]
    industry_group = row["industry_group"]
    
    update_query = "UPDATE industry SET industry_group = %s WHERE industry = %s"
    update_values = (industry_group, industry)
    cursor.execute(update_query, update_values)
    

mydb.commit()
cursor.close()
mydb.close()